In [1]:
from common_preamble import *
from scipy import stats

Loading data from /Users/caseys/Work/data_analysis/ais_analysis_pipeline/data/in/data_v185_ais_bounds_v3_is_chandelier_v5.h5


Minimum AIS length: 37187.05055806807


In [7]:
plot_dir = plot_dir + '/truncated_alt'
if not os.path.exists(plot_dir):
    print('Making new truncated plot directory...')
    os.mkdir(plot_dir)

#### Set the AIS length to analyze

In [8]:
from ais_synapse_utils import aggregate_ais_dataframes
complete_ais_ids = np.unique(ais_synapse_data['post_pt_root_id'])
ais_id_to_analyze = np.unique(ais_synapse_data['post_pt_root_id'])

ais_synapse_filter = ais_synapse_data['d_top_skel'] < min_ais_len
ais_synapse_data_f = ais_synapse_data[ais_synapse_filter]

aggregated_ais_syn_df = aggregate_ais_dataframes(complete_ais_ids, ais_synapse_data_f)
aggregated_ais_syn_df = aggregated_ais_syn_df[aggregated_ais_syn_df['ais_len'] >= min_ais_len].reset_index()

	Computing aggregate AIS information


In [10]:
in_analysis_set = np.isin(aggregated_ais_syn_df['post_pt_root_id'], list(ais_id_to_analyze))
aggregated_ais_syn_df = aggregated_ais_syn_df[in_analysis_set]

sv_df = dl.query_cell_types('soma_valence')
aggregated_ais_syn_df = aggregated_ais_syn_df.merge(sv_df[['pt_root_id', 'pt_position']].rename(columns={'pt_position':'soma_position'}),
                                                    how='left', left_on='post_pt_root_id', right_on='pt_root_id').drop(columns=('pt_root_id'))

soma_to_null = aggregated_ais_syn_df['soma_position'].map(lambda x: np.all(np.isnan(x)))
for ii in np.flatnonzero(soma_to_null):
    aggregated_ais_syn_df.at[ii, 'soma_position'] = np.full(3, np.nan)
    
aggregated_ais_syn_df['soma_x'] = np.vstack(aggregated_ais_syn_df['soma_position'].values)[:,0] * 4
aggregated_ais_syn_df['soma_y'] = np.vstack(aggregated_ais_syn_df['soma_position'].values)[:,1] * 4
aggregated_ais_syn_df['soma_z'] = np.vstack(aggregated_ais_syn_df['soma_position'].values)[:,2] * 40

aggregated_ais_syn_df['soma_x_um'] = np.vstack(aggregated_ais_syn_df['soma_position'].values)[:,0] * 4 / 1000
aggregated_ais_syn_df['soma_y_um'] = np.vstack(aggregated_ais_syn_df['soma_position'].values)[:,1] * 4 / 1000
aggregated_ais_syn_df['soma_z_um'] = np.vstack(aggregated_ais_syn_df['soma_position'].values)[:,2] * 40 / 1000

KeyError: 'soma_position'

Get a list of all synapses from AIS-targeting neurons onto our neurons in question

In [11]:
all_syn_filename = base_dir + '/data/in/all_synapses_onto_pycs_v{}.h5'.format(data_version)
if os.path.exists(all_syn_filename):
    all_post_synapses_df = pd.read_hdf(all_syn_filename, 'all_post_synapses_df')
else:
    syn_dfs = []
    for oid in tqdm.tqdm(np.unique(aggregated_ais_syn_df['post_pt_root_id'])):
        syn_dfs.append(dl.query_synapses(synapse_table, post_ids=[oid]))
    all_post_synapses_df = pd.concat(syn_dfs)
    all_post_synapses_df.to_hdf(all_syn_filename, 'all_post_synapses_df')
all_post_synapses_dec_df = all_post_synapses_df.merge(ais_synapse_data[['id', 'is_chandelier']], how='left', on='id')
non_ais_post_syns = all_post_synapses_dec_df[all_post_synapses_dec_df['is_chandelier'].map(np.isnan)]

In [12]:
full_soma_ids = dl.query_cell_ids('ais_analysis_soma')['pt_root_id'].values
arbor_ais_df = aggregated_ais_syn_df[np.isin(aggregated_ais_syn_df['post_pt_root_id'], full_soma_ids)]

NameError: name 'dl' is not defined

---
### Boutique soma fixing

In [15]:
original_data_version = 174
data_dir = base_dir + '/data/in'
soma_filename = data_dir + '/soma_metrics_v{}.h5'.format(original_data_version)

if os.path.exists(soma_filename):
    soma_synapse_df = pd.read_hdf(soma_filename, 'soma_metrics')
else:
    print('No soma data for this version!')

In [ ]:
soma_df = dl.query_cell_types(soma_table)

curr_root_ids = []
for _, row in soma_synapse_df.iterrows():
    curr_root_id = soma_df[soma_df['id']==row['id_soma']]['pt_root_id'].values[0]
    curr_root_ids.append(curr_root_id)
soma_synapse_df['curr_root_id'] = curr_root_ids

rel_columns = ['curr_root_id', 'label', 'mean_soma_syn_size', 'soma_area', 'soma_synapses', 'soma_syn_density']
arbor_ais_df = arbor_ais_df.merge(soma_synapse_df[rel_columns], how='left', left_on='post_pt_root_id', right_on='curr_root_id')

In [ ]:
fig, ax = plt.subplots(figsize=(2,2))
row_filter = arbor_ais_df['label']>1
sns.scatterplot(x='soma_synapses', y='soma_syn_density', data=arbor_ais_df[row_filter], ax=ax, color='k', alpha=0.5)
ax.set_xlabel('# Soma Syn.')
ax.set_ylabel('# Soma Syn./Soma Area ($\mu m^2$)')
sns.despine(ax=ax, offset=5)
fig.savefig(plot_dir + '/soma_syn_vs_soma_syn_density.pdf', bbox_inches='tight')

In [ ]:
arbor_ais_df['syn_net_chc'].std() / arbor_ais_df['syn_net_chc'].mean()

In [ ]:
arbor_ais_df['soma_synapses'].std() / arbor_ais_df['soma_synapses'].mean()

In [ ]:
arbor_ais_df['syn_net_chc'].describe()

In [ ]:
arbor_ais_df['soma_synapses'].describe()

### Rotate the data

In [ ]:
Rtrans = np.load(base_dir + '/data/in/pinky_rotation.npy')

In [ ]:
voxel_resolution

In [ ]:
soma_position_orig = np.vstack(arbor_ais_df['soma_position'].values) * voxel_resolution
soma_position_rot = np.dot(Rtrans, soma_position_orig.T).T
rot_offset = np.min(soma_position_rot, axis=0)
soma_position_rot = soma_position_rot - rot_offset

arbor_ais_df['soma_position_rot'] = [xyz for xyz in soma_position_rot]
arbor_ais_df['soma_x'] = soma_position_rot[:,0]
arbor_ais_df['soma_y'] = soma_position_rot[:,1]
arbor_ais_df['soma_z'] = soma_position_rot[:,2]

arbor_ais_df['soma_x_um'] = soma_position_rot[:,0] / 1000
arbor_ais_df['soma_y_um'] = soma_position_rot[:,1] / 1000
arbor_ais_df['soma_z_um'] = soma_position_rot[:,2] / 1000

---
Get AIS radius

In [ ]:
from itertools import product
from collections import defaultdict
from meshparty import skeletonize, skeleton_io

mm = trimesh_io.MeshMeta(disk_cache_path=mesh_dir,
                         cache_size=0, cv_path=mesh_cv_path, voxel_scaling=voxel_scaling)

ais_meshes = []
for oid in complete_ais_ids:
    ais_file = ais_mesh_dir + '/{}_ais.h5'.format(oid)
    if os.path.exists(ais_file):
        ais_mesh = mm.mesh(filename=ais_file)
        ais_meshes.append(ais_mesh)
    else:
        ais_meshes.append(None)
        print('{} AIS not found!'.format(oid))

data_dir = base_dir + '/data'

ais_sks = []
for oid, ais_mesh in zip(complete_ais_ids, ais_meshes):
    ais_sk_filename = skel_dir + '/sk_ais_{}.h5'.format(oid)
    if os.path.exists(ais_sk_filename):
        ais_sks.append(skeleton_io.read_skeleton_h5(ais_sk_filename))
    else:
        ais_sk = skeletonize.skeletonize_mesh(ais_mesh )
        ais_sks.append(ais_sk)
        skeleton_io.write_skeleton_h5(ais_sk, ais_sk_filename)

In [ ]:
d, inds = ais_meshes[0].kdtree.query( ais_sks[0].vertices )

In [ ]:
from meshparty import ray_tracing
ais_rad = []
for m, sk in zip(ais_meshes, ais_sks):
    d, inds = m.kdtree.query( sk.vertices )
    rs = ray_tracing.ray_trace_distance(inds, m)
    ds = sk.distance_to_root
    med_r = np.nanmedian(rs[ np.logical_and(ds>initial_cutoff_length, ds<min_ais_len)])/2
    ais_rad.append(med_r)

In [ ]:
ais_rad

In [ ]:
initial_cutoff_length = 5000
ais_radius = []
for ais_sk in ais_sks:
    rs= np.array(ais_sk.vertex_properties['rs'][:-1])
    ais_radius.append(np.nanmedian(rs[np.logical_and(ais_sk.distance_to_root< min_ais_len, ais_sk.distance_to_root>initial_cutoff_length)])/2)
ais_radius_df = pd.DataFrame(data={'post_pt_root_id': complete_ais_ids, 'ais_radius': ais_radius})

In [ ]:
arbor_ais_df = arbor_ais_df.merge(ais_radius_df, on='post_pt_root_id')
plot_label_lookup['ais_radius'] = 'AIS radius (nm)'


---

In [ ]:
chc_syn_df = dl.query_synapses(synapse_table, pre_ids=chc_ids)
nonchc_syn_df = dl.query_synapses(synapse_table, pre_ids=np.unique(ais_synapse_data.query('is_chandelier==False')['pre_pt_root_id']))

chc_syn_loc = np.vstack(chc_syn_df['ctr_pt_position'].values) * voxel_resolution
nonchc_syn_loc = np.vstack(nonchc_syn_df['ctr_pt_position'].values) * voxel_resolution

chc_syn_loc_rot = np.dot(Rtrans, chc_syn_loc.T).T - rot_offset
nonchc_syn_loc_rot = np.dot(Rtrans, nonchc_syn_loc.T).T - rot_offset

In [ ]:
bwidth = 20
bins = np.arange(-40, 200, bwidth)
chc_depth_ct, _ = np.histogram(chc_syn_loc_rot[:,1] / 1000, bins=bins)
non_depth_ct, _ = np.histogram(nonchc_syn_loc_rot[:,1] / 1000, bins=bins)

In [ ]:
figsize = (5,8)

alldat = np.concatenate((soma_position_rot, chc_syn_loc_rot, nonchc_syn_loc_rot)) / 1000
xax_min, yax_min, zax_min = np.min(alldat, axis=0).tolist()
xax_max, yax_max, zax_max = np.max(alldat, axis=0).tolist()
hull_x, hull_y = convex_hull_xy(alldat[:, [0,1]])

yticks = np.arange(0,161,20)

fig, ax = plt.subplots(figsize=figsize)
ax.plot(nonchc_syn_loc_rot[:,0]/1000, nonchc_syn_loc_rot[:, 1]/1000, marker='.', linestyle='none', color=non_color, alpha=0.01)
ax.plot(hull_x, hull_y, color=non_color, alpha=0.5)
ax.set_aspect(1)
ax.set_xlim([xax_min, xax_max])
ax.set_ylim([yax_min, yax_max])
ax.set_yticks(yticks)
ax.invert_yaxis()
ax.set_title('Non-ChC Synapses (n={})'.format(len(nonchc_syn_loc_rot)))
sns.despine(offset=5, ax=ax, trim=True)
fig.savefig(fname=plot_dir+'/synapse_all_nonchc.pdf', dpi=300, bbox_inches="tight")

fig, ax = plt.subplots(figsize=figsize)
ax.plot(chc_syn_loc_rot[:,0]/1000, chc_syn_loc_rot[:, 1]/1000, marker='.', linestyle='none', color=chc_color, alpha=0.2)
ax.plot(hull_x, hull_y, color=non_color, alpha=0.5)
ax.set_aspect(1)
ax.set_xlim([xax_min, xax_max])
ax.set_ylim([yax_min, yax_max])
ax.set_yticks(yticks)
ax.invert_yaxis()
ax.set_title('ChC Synapses (n={})'.format(len(chc_syn_loc_rot)))
sns.despine(offset=5, ax=ax, trim=True)
fig.savefig(fname=plot_dir+'/synapse_all_chc.pdf', dpi=300, bbox_inches="tight")

fig, ax = plt.subplots(figsize=figsize)
ax.plot(soma_position_rot[:,0]/1000, soma_position_rot[:, 1]/1000, marker='o', linestyle='none',
           color='k', alpha=0.8, markersize=6, markeredgecolor='w')
ax.plot(hull_x, hull_y, color=non_color, alpha=0.5)
ax.set_aspect(1)
ax.set_xlim([xax_min, xax_max])
ax.set_ylim([yax_min, yax_max])
ax.set_yticks(yticks)
ax.invert_yaxis()
ax.set_title('Soma')
sns.despine(offset=5, ax=ax, trim=True)
fig.savefig(fname=plot_dir+'/synapse_all_soma.pdf', dpi=300, bbox_inches="tight")

In [ ]:
figsize = (5,8)

alldat = np.concatenate((soma_position_rot, chc_syn_loc_rot, nonchc_syn_loc_rot)) / 1000
xax_min, yax_min, zax_min = np.min(alldat, axis=0).tolist()
xax_max, yax_max, zax_max = np.max(alldat, axis=0).tolist()
hull_x, hull_y = convex_hull_xy(alldat[:, [0,1]])

yticks = np.arange(0,161,20)

fig, ax = plt.subplots(figsize=figsize)
ax.plot(nonchc_syn_loc_rot[:,0]/1000, nonchc_syn_loc_rot[:, 1]/1000, marker='.', linestyle='none', color=non_color, alpha=0.015)
ax.plot(chc_syn_loc_rot[:,0]/1000, chc_syn_loc_rot[:, 1]/1000, marker='.', linestyle='none', color=chc_color, alpha=0.15)
ax.plot(hull_x, hull_y, color=non_color, alpha=0.5)
ax.set_aspect(1)
ax.set_xlim([xax_min, xax_max])
ax.set_ylim([yax_min, yax_max])
ax.set_yticks(yticks)
ax.invert_yaxis()
sns.despine(offset=5, ax=ax, trim=True)
fig.savefig(fname=plot_dir+'/synapse_all_chc_plus_nonchc.pdf', bbox_inches="tight")

In [ ]:
stars = [0.05, 0.01, 0.001]

bwidth = 10
bins = np.arange(0, 150, bwidth)
chc_depth_ct, _ = np.histogram(chc_syn_loc_rot[:,1] / 1000, bins=bins)
non_depth_ct, _ = np.histogram(nonchc_syn_loc_rot[:,1] / 1000, bins=bins)

fig, ax = plt.subplots(figsize=(6,4), ncols=3)

ax[0].grid(True, axis='x')
ax[0].set_axisbelow(True)
ax[0].hist(chc_syn_loc_rot[:,1] / 1000, bins=bins, orientation='horizontal', color=chc_color, edgecolor='w')
ax[0].set_ylim(bins[0], bins[-1])
ax[0].invert_yaxis()
ax[0].set_ylabel('Depth ($\mu m$)')
ax[0].set_xlabel('# Syn ChC')

ax[1].grid(True, axis='x', which='both')
ax[1].set_axisbelow(True)
ax[1].hist(nonchc_syn_loc_rot[:,1] / 1000, bins=bins, orientation='horizontal', color=non_color, edgecolor='w')
ax[1].set_ylim(bins[0], bins[-1])
ax[1].invert_yaxis()
ax[1].set_xlabel('# Syn Non-ChC')

ax[2].plot(chc_depth_ct / (non_depth_ct + chc_depth_ct), bins[0:-1]+bwidth/2, 'o:', color=chc_color, linewidth=1)

pvals = []
for chc_ct, non_ct in zip(chc_depth_ct, non_depth_ct):
    v, p = stats.fisher_exact([[chc_ct, non_ct], [sum(chc_depth_ct), sum(non_depth_ct)]])
    pvals.append(p)
_, pval_corr, _, _ = multitest.multipletests(pvals, method='hs')
n_stars = assign_stars(pval_corr, stars)
plot_stars(chc_depth_ct / (non_depth_ct + chc_depth_ct),
           bins[0:-1]+bwidth/2+4, n_stars, ax[2])

ax[2].grid(True, axis='y')
ax[2].set_axisbelow(True)

total_fraction = np.sum(chc_depth_ct) / np.sum(chc_depth_ct + non_depth_ct)
ax[2].plot([total_fraction, total_fraction], [bins[0], bins[-1]], 'k--')

ax[2].set_xlim(-0.01, 0.08)
ax[2].set_ylim(bins[0], bins[-1])
ax[2].set_xticks(np.arange(0, 0.081,0.04))
ax[2].invert_yaxis()
ax[2].set_xlabel('Fraction Syn ChC')

sns.despine(offset=5, ax=ax[0])
sns.despine(offset=5, ax=ax[1])
sns.despine(offset=5, ax=ax[2])
plt.tight_layout()

fig.savefig(fname=plot_dir+'/synapse_depth_distribution.pdf', bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), ncols=2)
ax[0].plot(chc_syn_loc_rot[:,2]/1000, chc_syn_loc_rot[:, 1]/1000, marker='.', linestyle='none', color=chc_color, alpha=0.1)
ax[0].set_aspect(1)
ax[0].invert_yaxis()
ax[0].set_title('ChC Syns')
sns.despine(offset=5, ax=ax[0])

ax[1].plot(nonchc_syn_loc_rot[:,2]/1000, nonchc_syn_loc_rot[:, 1]/1000, marker='.', linestyle='none', color=non_color, alpha=0.002)
ax[1].set_aspect(1)
ax[1].invert_yaxis()
ax[1].set_title('Non-ChC Syns')
sns.despine(offset=5, ax=ax[1])

fig.savefig(fname=plot_dir+'/synapse_side_views.pdf', bbox_inches="tight")

In [ ]:
def nan_pearsonr(x,y):
    good_data_x = np.logical_and(~np.isnan(x), ~np.isinf(x))
    good_data_y = np.logical_and(~np.isnan(y), ~np.isinf(y))
    good_data = np.logical_and(good_data_x, good_data_y)
    return stats.pearsonr(x[good_data], y[good_data])

In [ ]:
figsize = (2, 2)

arbor_ais_df['soma_y_adj'] = arbor_ais_df['soma_y_um']
common_y = ['soma_y_adj', 'n_syn_soma', 'soma_x_um']
if 'label' in arbor_ais_df.columns:
    common_y += ['soma_synapses', 'soma_area', 'soma_syn_density', 'ais_radius']

col_pairs = {'syn_net_chc': ['syn_net_non', 'num_cells_chc', 'syn_mean_chc'] + common_y,
             'syn_net_non': ['syn_net_chc'] + common_y,
             'syn_mean_chc': ['num_cells_chc'] + common_y,
             'num_cells_chc': ['syn_mean_chc'] + common_y,
             }
invert_columns = ['soma_y_adj']
tickintdict = {'soma_syn_density': False, 'syn_mean_chc': False}

tick_dict = {'syn_net_chc': np.arange(0,27,5),
             'syn_net_non': np.arange(0,27,5),
             'soma_y_adj': np.arange(0, 121, 20),
             'soma_x_um': np.arange(0, 251, 50),
             'n_syn_soma': np.arange(60,161,20),
             'soma_synapses': np.arange(40, 121, 20),
             'soma_area': np.arange(450, 801, 100),
             'soma_syn_density': np.arange(0.06, 0.181, 0.02),
             'num_cells_chc': np.arange(0,10,2),
             'syn_mean_chc': np.arange(0,8.1,2),
             'ais_radius': np.arange(175, 376, 50)}

xprecision = {'soma_syn_density': 2}

needs_label = ['soma_synapses', 'soma_area', 'soma_syn_density']
for y in col_pairs:
    for x in col_pairs.get(y):
        if x in needs_label or y in needs_label:
            use_df = arbor_ais_df[row_filter]
        else:
            use_df = arbor_ais_df
        fig, ax = make_scatterplot(x, y, use_df,
                                   figsize, plot_label_lookup, tick_dict,
                                   xtick_int=tickintdict.get(x, True),
                                   ytick_int=tickintdict.get(y, True),
                                   xprecision=xprecision.get(x, 1))
        ax.set_title(f'R = {nan_pearsonr(use_df[x], use_df[y])[0]:.2f}')
        fig.savefig(fname=plot_dir+'/scatterplots_{y}_vs_{x}.pdf'.format(x=x, y=y), bbox_inches="tight")

In [ ]:
row_filter = (arbor_ais_df['label']>1) & (~np.isnan(arbor_ais_df['soma_area']))

In [ ]:
corr_matrix_columns = [
 'soma_synapses',
 'soma_area',
 'soma_syn_density',
 'syn_net_non',
 'ais_radius',
 'soma_y_um']

extra_corr_matrix_columns = ['syn_net_chc']+corr_matrix_columns
ais_item_data = arbor_ais_df[row_filter][extra_corr_matrix_columns]

corr_mat = np.zeros((len(extra_corr_matrix_columns), len(extra_corr_matrix_columns)))
corr_mat_p = np.zeros((len(extra_corr_matrix_columns), len(extra_corr_matrix_columns)))
for ii in range(len(extra_corr_matrix_columns)):
    for jj in range(len(extra_corr_matrix_columns)):
        r, p = nan_pearsonr(ais_item_data[extra_corr_matrix_columns[ii]], ais_item_data[extra_corr_matrix_columns[jj]])
        corr_mat[ii,jj] = r
        corr_mat_p[ii,jj] = p

In [ ]:
from statsmodels.stats import multitest
is_sig, corr_p, _, _ = multitest.multipletests(corr_mat_p[np.tril_indices_from(corr_mat, k=-1)])
tri_inds = np.tril_indices_from(corr_mat, k=-1)
put_star = []
for ii, jj, sig in zip(*tri_inds, is_sig):
    if sig:
        put_star.append([ii, jj])
put_star = np.array(put_star)

In [ ]:
do_plot = False 

mask = np.zeros_like(corr_mat, dtype=np.bool)
mask[np.tril_indices_from(mask)] = True

cmap = sns.color_palette('RdBu', n_colors=31)
fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(corr_mat, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)
ax.plot(put_star[:,0]+0.5, put_star[:,1]+0.5, 'k*')
ax.set_yticks(np.arange(0.5, len(extra_corr_matrix_columns)-1+0.5))
_=ax.set_yticklabels([plot_label_lookup[x] for x in extra_corr_matrix_columns[:-1]], rotation=0)

ax.set_xticks(np.arange(1.5, len(extra_corr_matrix_columns)+0.5))
_=ax.set_xticklabels([plot_label_lookup[x] for x in extra_corr_matrix_columns[1:]], rotation=45)
ax.xaxis.tick_top()
if do_plot is True:
    fig.savefig(f'{plot_dir}/variable_correlation_structure.pdf', bbox_inches='tight')

In [ ]:
from sklearn.decomposition import *

ais_item_data = arbor_ais_df[row_filter][corr_matrix_columns]
Xdat = ais_item_data.values
Xz = stats.zscore(Xdat, axis=0)

# pca = SparsePCA(n_components=3, random_state=100 )
pca = FastICA(n_components=3, random_state=1004 )

Xz_pca = pca.fit_transform(Xz)

for ii in range(pca.n_components):
    if pca.components_[ii,np.argmax(np.abs(pca.components_[ii,:]))] < 0:   # If the dominant component is negative
        pca.components_[ii, :] = -1 * pca.components_[ii, :]
        Xz_pca[:,ii] = -1 * Xz_pca[:,ii]

do_plot = True 
for ii in range(pca.n_components):
    fig, ax = plt.subplots(figsize=(1,2))
    ax.barh(np.arange(pca.components_.shape[1]), pca.components_[ii,:], height=0.5, color='k')
    ax.vlines(0, -1, 6, linewidth=1)
    ax.set_ylim((-0.5, 5.5))
    ax.set_yticks(np.arange(pca.components_.shape[1]))
    ax.set_yticklabels([plot_label_lookup[x] for x in corr_matrix_columns])
    ax.invert_yaxis()
    maxval=np.max(np.abs(pca.components_[ii,:]))
    ax.set_xlim((-maxval, maxval))
    ax.set_xticks((-maxval, 0, maxval))
    ax.set_xticklabels((f'{-maxval:0.2f}', '0', f'{maxval:0.2f}'))
    sns.despine(ax=ax, offset=5, trim=True)
    ax.set_title(f'PC {ii}')
    if do_plot:
        fig.savefig(f'{plot_dir}/factor_component_{ii}.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
sns.scatterplot(x=Xz_pca[:, 0], y=Xz_pca[:,1], hue=Xz_pca[:,2], s=70,  palette='viridis', legend='brief')
ax.set_xlabel('"Non-ChC Input"', fontdict={'size':14})
ax.set_ylabel('"Soma Size"', fontdict={'size':14})
ax.legend(bbox_to_anchor=(1.1, 1))
sns.despine(ax=ax, offset=5)

fig.savefig(f'{plot_dir}/post_ica_scatter.pdf', bbox_inches='tight')

In [ ]:
arbor_ais_df_use = arbor_ais_df[row_filter]

In [ ]:
for ii in np.arange(pca.n_components):
    arbor_ais_df_use[f'pca_{ii}'] = Xz_pca[:,ii]

In [ ]:
plot_label_lookup = {'syn_net_chc': '# ChC Syn.',
                     'syn_net_non': '# Non-ChC Syn.',
                     'size_net_chc': 'Net ChC Syn. Size',
                     'soma_y_adj': 'Soma Depth ($\mu m$)',
                     'soma_y_um': 'Soma Depth ($\mu m$)',
                     'soma_x_um': 'Soma Mediolateral Pos. ($\mu m$)',
                     'n_syn_soma': '# Syn Soma',
                     'soma_synapses': '# Syn Soma',
                     'soma_area': 'Soma Area ($\mu m^2$)',
                     'soma_syn_density': '# Syn Soma/($\mu m^2$)',
                     'num_cells_chc': '# ChC Connections',
                     'syn_mean_chc': '# Syn/Connection',
                     'conn_frac': 'Connectivity Fraction',
                     'num_potential': '# Potential ChC',
                     'size_mean_chc': 'Mean ChC Syn Size',
                     'pca_2': 'Soma Size Comp.',
                     'pca_0': 'Soma Depth Comp.',
                     'pca_1': 'Soma Inhibition Comp.',
                     'pca_3': 'PC 3',
                     'pca_4': 'PC 4',
                     'ais_radius': 'AIS Radius',
                     'syn_max_chc': 'Max Syn ChC',
                    }

In [ ]:
row_filter_true = arbor_ais_df_use['post_pt_root_id']>0

In [ ]:
do_save= True 
ls_prefix = {True: 'rls', False: 'ols'}
for use_robust in [True, False]:
    ### Same for good soma cutout
    base_variables = [f'pca_{ii}' for ii in range(pca.n_components)]

    y_col = 'syn_net_chc'
    columns_chc = base_variables
    fig, ax, res_df = ols_analysis_single(arbor_ais_df_use, row_filter_true, 'syn_net_chc', columns_chc[::-1], chc_color, robust=use_robust, plot_label_lookup=plot_label_lookup, xticks=np.arange(-0.75,.76,0.25))
    if do_save:
        res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_factor_analysis_{y_col}.csv'.format(y_col), index=False)
        fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_factor_analysis_{y_col}.pdf'.format(y_col), bbox_inches="tight")
        residual_scatterplots(y_col, columns_chc, row_filter_true, arbor_ais_df_use, 'exact', plot_dir, plot_label_lookup=plot_label_lookup, robust=use_robust)
    
    y_col = 'size_net_chc'
    columns_chc = base_variables
    fig, ax, res_df = ols_analysis_single(arbor_ais_df_use, row_filter_true, y_col, columns_chc[::-1], chc_color, robust=use_robust, plot_label_lookup=plot_label_lookup, xticks=np.arange(-0.75,.76,0.25))
    if do_save:
        res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_factor_analysis_{y_col}.csv'.format(y_col), index=False)
        fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_factor_analysis_{y_col}.pdf'.format(y_col), bbox_inches="tight")
        residual_scatterplots(y_col, columns_chc, row_filter_true, arbor_ais_df_use, 'exact', plot_dir, plot_label_lookup=plot_label_lookup, robust=use_robust)

    y_col = 'syn_mean_chc'
    columns_mean = base_variables
    fig, ax, res_df = ols_analysis_single(arbor_ais_df_use, row_filter_true, y_col, columns_mean[::-1], syn_per_conn_color, robust=use_robust, plot_label_lookup=plot_label_lookup, xticks=np.arange(-0.75,.76,0.25))
    if do_save:
        res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_factor_analysis_{y_col}.csv'.format(y_col), index=False)
        fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_factor_analysis_{y_col}.pdf'.format(y_col), bbox_inches="tight")
        residual_scatterplots(y_col, columns_chc, row_filter_true, arbor_ais_df_use, 'exact', plot_dir, plot_label_lookup=plot_label_lookup, robust=use_robust)

    y_col = 'num_cells_chc'
    columns_num = base_variables
    fig, ax, res_df = ols_analysis_single(arbor_ais_df_use, row_filter_true, y_col, columns_num[::-1], num_conn_color, robust=use_robust, plot_label_lookup=plot_label_lookup, xticks=np.arange(-0.75,.76,0.25))
    if do_save:
        res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_factor_analysis_{y_col}.csv'.format(y_col), index=False)
        fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_factor_analysis_{y_col}.pdf'.format(y_col), bbox_inches="tight")
        residual_scatterplots(y_col, columns_chc, row_filter_true, arbor_ais_df_use, 'exact', plot_dir, plot_label_lookup=plot_label_lookup, robust=use_robust)


In [ ]:
for ycol in ['syn_net_chc', 'syn_mean_chc', 'num_cells_chc']:
    for xcol in base_variables:
        for use_robust in [True, False]:
            fig, ax = plt.subplots(figsize=(3,3))
            sns.regplot(x=xcol, y=ycol, data=arbor_ais_df_use, ax=ax, marker='o', color='k', scatter_kws={'s':8, 'color':(0.3, 0.3, 0.3)}, robust=use_robust, n_boot=100)
            sns.despine(ax=ax, offset=5)
            ax.set_xlabel(plot_label_lookup[xcol])
            ax.set_ylabel(plot_label_lookup[ycol])
            r=nan_pearsonr(arbor_ais_df_use[xcol], arbor_ais_df_use[ycol])[0]
            ax.set_title(f'R$^2$={r*r:.2f}')
            fig.savefig(f'{plot_dir}/linear_plot_robust_{use_robust}_{ycol}_vs_{xcol}.pdf')

In [ ]:
# ls_prefix = {True: 'rls', False: 'ols'}
# for use_robust in [True, False]:
#     ### Same for good soma cutout
#     base_variables = corr_matrix_columns

#     y_col = 'syn_net_chc'
#     columns_chc = ['syn_net_non'] + base_variables
#     fig, ax, res_df = ols_analysis_single(arbor_ais_df_use, row_filter_true, 'syn_net_chc', columns_chc[::-1], chc_color, robust=use_robust, plot_label_lookup=plot_label_lookup,  xticks=np.arange(-0.75,.76,0.25))
#     res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_exact_{y_col}.csv'.format(y_col), index=False)
#     fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_exact_{y_col}.pdf'.format(y_col), bbox_inches="tight")
#     residual_scatterplots(y_col, columns_chc, row_filter, arbor_ais_df, 'exact', plot_dir,  plot_label_lookup=plot_label_lookup,robust=robust)

#     y_col = 'size_net_chc'
#     columns_chc = ['syn_net_non'] + base_variables
#     fig, ax, res_df = ols_analysis_single(arbor_ais_df_use, row_filter_true, 'size_net_chc', columns_chc[::-1], chc_color, robust=use_robust, plot_label_lookup=plot_label_lookup,  xticks=np.arange(-0.75,.76,0.25))
#     res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_exact_{y_col}.csv'.format(y_col), index=False)
#     fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_exact_{y_col}.pdf'.format(y_col), bbox_inches="tight")
#     residual_scatterplots(y_col, columns_chc, row_filter, arbor_ais_df, 'exact', plot_dir,  plot_label_lookup=plot_label_lookup,robust=robust)
    
#     y_col = 'syn_net_non'
#     columns_non = ['syn_net_chc'] + base_variables
#     fig, ax, res_df = ols_analysis_single(arbor_ais_df_use, row_filter_true, y_col, columns_non[::-1], non_color, robust=use_robust, plot_label_lookup=plot_label_lookup, xticks=np.arange(-0.75,.76,0.25))
#     res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_exact_{y_col}.csv'.format(y_col), index=False)
#     fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_exact_{y_col}.pdf'.format(y_col), bbox_inches="tight")
#     residual_scatterplots(y_col, columns_non, row_filter, arbor_ais_df, 'exact', plot_dir,  plot_label_lookup=plot_label_lookup,robust=robust)

#     y_col = 'syn_mean_chc'
#     columns_mean = ['syn_net_non'] + base_variables
#     fig, ax, res_df = ols_analysis_single(arbor_ais_df_use, row_filter_true, y_col, columns_mean[::-1], syn_per_conn_color, robust=use_robust, plot_label_lookup=plot_label_lookup, xticks=np.arange(-0.75,.76,0.25))
#     res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_exact_{y_col}.csv'.format(y_col), index=False)
#     fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_exact_{y_col}.pdf'.format(y_col), bbox_inches="tight")
#     residual_scatterplots(y_col, columns_mean, row_filter, arbor_ais_df, 'exact', plot_dir,  plot_label_lookup=plot_label_lookup,robust=robust)

#     y_col = 'num_cells_chc'
#     columns_num = ['syn_net_non'] + base_variables
#     fig, ax, res_df = ols_analysis_single(arbor_ais_df_use, row_filter_true, y_col, columns_num[::-1], num_conn_color, robust=use_robust, xticks=np.arange(-0.75,.76,0.25))
#     res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_exact_{y_col}.csv'.format(y_col), index=False)
#     fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_exact_{y_col}.pdf'.format(y_col), bbox_inches="tight")
#     residual_scatterplots(y_col, columns_num, row_filter, arbor_ais_df, 'exact', plot_dir,  plot_label_lookup=plot_label_lookup,robust=robust)


In [ ]:
#### Same for soma synapse density
for use_robust in [True, False]:

    columns_chc = ['soma_syn_density', 'syn_net_non', 'soma_y_um', 'soma_x_um', 'ais_radius']
    y_col = 'syn_net_chc'
    fig, ax, res_df = ols_analysis_single(arbor_ais_df_use, row_filter_true, 'syn_net_chc', columns_chc[::-1], chc_color, robust=use_robust,  xticks=np.arange(-0.75,.76,0.25))
    res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_density_{y_col}.csv'.format(y_col), index=False)
    fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_density_{y_col}.pdf'.format(y_col), bbox_inches="tight")
    residual_scatterplots(y_col, columns_chc, row_filter, arbor_ais_df, 'density', plot_dir, robust=robust)

    columns_non = ['soma_syn_density', 'syn_net_chc', 'soma_y_um', 'soma_x_um', 'ais_radius']
    y_col = 'syn_net_non'
    fig, ax, res_df = ols_analysis_single(arbor_ais_df_use, row_filter_true, y_col, columns_non[::-1], non_color, robust=use_robust, xticks=np.arange(-0.75,.76,0.25))
    res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_density_{y_col}.csv'.format(y_col), index=False)
    fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_density_{y_col}.pdf'.format(y_col), bbox_inches="tight")
    residual_scatterplots(y_col, columns_non, row_filter, arbor_ais_df, 'density', plot_dir, robust=robust)

    y_col = 'syn_mean_chc'
    columns_mean = ['soma_syn_density', 'syn_net_non', 'soma_y_um', 'soma_x_um', 'ais_radius']
    fig, ax, res_df = ols_analysis_single(arbor_ais_df_use, row_filter_true, y_col, columns_mean[::-1], syn_per_conn_color, robust=use_robust, xticks=np.arange(-0.75,.76,0.25))
    res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_density_{y_col}.csv'.format(y_col), index=False)
    fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_density_{y_col}.pdf'.format(y_col), bbox_inches="tight")
    residual_scatterplots(y_col, columns_mean, row_filter, arbor_ais_df, 'density', plot_dir, robust=robust)

    y_col = 'num_cells_chc'
    columns_num = ['soma_syn_density', 'syn_net_non', 'soma_y_um', 'soma_x_um', 'ais_radius']
    fig, ax, res_df = ols_analysis_single(arbor_ais_df_use, row_filter_true, y_col, columns_num[::-1], num_conn_color, robust=use_robust, xticks=np.arange(-0.75,.76,0.25))
    res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_density_{y_col}.csv'.format(y_col), index=False)
    fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_soma_density_{y_col}.pdf'.format(y_col), bbox_inches="tight")
    residual_scatterplots(y_col, columns_num, row_filter, arbor_ais_df, 'density', plot_dir, robust=robust)


---

---
### Spatial considerations

In [ ]:
from meshparty import skeletonize, mesh_filters
from scipy import sparse

In [ ]:
soma_ball_radius = 10000
axon_component_thresh = 0.00018 # synapse vertices per component vertex

chc_soma_df = soma_df.query(f'pt_root_id in {chc_ids.tolist()}')
chc_soma_ids = chc_soma_df['pt_root_id'].values
chc_soma_pts = np.vstack(chc_soma_df['pt_position'].values) * voxel_resolution

for chc_oid in chc_soma_ids:
    if not os.path.exists(f'{mesh_dir}/{chc_oid}_axon.h5'):
        print(f'Extracting axon for {chc_oid}...')
        chc_soma_mesh = mm.mesh(seg_id = chc_oid)
        chc_soma_mesh.add_link_edges(dataset_name='pinky100', seg_id=chc_oid)
        chc_soma_mesh = chc_soma_mesh.apply_mask(mesh_filters.filter_components_by_size(chc_soma_mesh, min_size=1000))

        out_syn_df = dl.query_synapses(synapse_table, pre_ids=[chc_oid])
        out_syn_pts = np.vstack(out_syn_df['ctr_pt_position'].values) * voxel_resolution

        ds, minds = chc_soma_mesh.kdtree.query(out_syn_pts, distance_upper_bound=250)
        minds = chc_soma_mesh.map_indices_to_unmasked(minds[~np.isinf(ds)])

        soma_ball = mesh_filters.filter_spatial_distance_from_points(chc_soma_mesh, chc_soma_pts, soma_ball_radius)
        chc_soma_mesh_nonsoma = chc_soma_mesh.apply_mask(~soma_ball)
        filtered_minds = chc_soma_mesh_nonsoma.filter_unmasked_indices(minds)

        _, comp_inds = sparse.csgraph.connected_components(chc_soma_mesh_nonsoma.csgraph)

        all_comp_ind, comp_vert_counts = np.unique(comp_inds, return_counts=True)
        pre_syn_comp_ind, pre_syn_counts = np.unique(comp_inds[filtered_minds], return_counts=True)

        pre_syn_counts_all = np.zeros(all_comp_ind.shape)
        pre_syn_counts_all[pre_syn_comp_ind] = pre_syn_counts
        axon_components = all_comp_ind[pre_syn_counts_all/comp_vert_counts > axon_component_thresh]

        axon_minds = np.isin(comp_inds, axon_components)
        chc_axon_mesh = chc_soma_mesh_nonsoma.apply_mask(axon_minds)
        chc_axon_mesh.write_to_file(f'{mesh_dir}/{chc_oid}_axon.h5')
    else:
        print(f'Already extracted axon for {chc_oid}')

In [ ]:
mm = trimesh_io.MeshMeta(disk_cache_path=mesh_dir,
                         cache_size=0, cv_path=mesh_cv_path, voxel_scaling=voxel_scaling)

ais_meshes = {} 
for oid in complete_ais_ids:
    ais_file = ais_mesh_dir + '/{}_ais.h5'.format(oid)
    if os.path.exists(ais_file):
        ais_mesh = mm.mesh(filename=ais_file, voxel_scaling=voxel_scaling)
        ais_meshes[oid] = ais_mesh
    else:
        ais_meshes[oid] = None
        print('{} AIS not found!'.format(oid))

chc_meshes = {}
for oid in chc_ids:
    if oid not in chc_soma_ids:
        chc_mesh = mm.mesh(seg_id=oid, voxel_scaling=voxel_scaling)
    else:
        chc_mesh = mm.mesh(filename=mesh_dir + '/{}_axon.h5'.format(oid), voxel_scaling=voxel_scaling)
    chc_meshes[oid] = chc_mesh

#### Use the multi-utilities to get potential synapses from meshes

In [ ]:
from itertools import product
from collections import defaultdict
from meshparty import skeletonize, skeleton_io

data_dir = base_dir + '/data'

from multiwrapper import multiprocessing_utils as mu
def _multi_count_neighbors(args):
    ais_kdt, chc_kdt, d_max = args
    return ais_kdt.count_neighbors(chc_kdt, d_max)

ais_sks = {} 
for oid in complete_ais_ids:
    ais_sk_filename = skel_dir + '/sk_ais_{}.h5'.format(oid)
    ais_sk = skeleton_io.read_skeleton_h5(ais_sk_filename)
    ais_sk.voxel_scaling = None
    ais_sks[oid] = ais_sk
    
# chc_sks = []
# for chc_oid in chc_meshes:
#     chc_sk_filename = skel_dir + '/sk_chc_axon_{}.h5'.format(chc_oid)
#     if os.path.exists(chc_sk_filename):
#         chc_sks.append(skeleton_io.read_skeleton_h5(chc_sk_filename))
#     else:
#         chc_sk = skeletonize.skeletonize_mesh(chc_meshes[chc_oid], invalidation_d=4000)
#         chc_sks.append(chc_sk)
#         skeleton_io.write_skeleton_h5(chc_sk, chc_sk_filename)

In [ ]:
d_vec = [5000, 7500, 10000, 15000]
is_potential_ds = {}
for d_max in d_vec:
    multi_args = []
    for ais_oid, chc_mesh in product(complete_ais_ids, chc_meshes.values()):
        ais_sk = ais_sks[ais_oid]
        fair_inds = ais_sk.distance_to_root < min_ais_len
        ais_sk_fair_verts = ais_sk.vertices[fair_inds]
        ais_fair_kdt = spatial.cKDTree(ais_sk_fair_verts)
        
        multi_args.append([ais_fair_kdt, chc_mesh.kdtree, d_max])    

    n_neib_list = mu.multiprocess_func(_multi_count_neighbors, multi_args, verbose=True)

    is_potential = defaultdict(dict)
    n_neib_list_copy = n_neib_list.copy()
    for ais_ind, chc_oid in product(range(len(ais_meshes)), chc_meshes.keys()):
        is_potential[ais_ind][chc_oid] = n_neib_list_copy.pop(0)>0
    is_potential_ds[d_max] = is_potential

In [ ]:
tick_dict = {'syn_net_chc': np.arange(0,27,5),
             'syn_net_non': np.arange(0,27,5),
             'soma_y_adj': np.arange(0, 121, 20),
             'soma_x_um': np.arange(0, 251, 50),
             'n_syn_soma': np.arange(60,161,20),
             'soma_synapses': np.arange(40, 121, 20),
             'soma_area': np.arange(500, 901, 100),
             'soma_syn_density': np.arange(0.05, 0.15, 0.02),
             'num_cells_chc': np.arange(0,10,2.5),
             'syn_mean_chc': np.arange(0,8,2),
             'conn_frac': np.arange(0,1.01,0.2),
             'num_potential': np.arange(0, 21, 5),
             }


In [ ]:
fraction_cutoff = 0.1
is_potential = is_potential_ds[10000]

num_pot_df = pd.DataFrame({'pyc_root_id': complete_ais_ids, 'num_potential':[sum(is_potential[ii].values()) for ii in range(len(complete_ais_ids))]})
arbor_ais_df_pot = arbor_ais_df_use.merge(num_pot_df, left_on='post_pt_root_id', right_on='pyc_root_id', how='left').drop(columns=['pyc_root_id'])
arbor_ais_df_pot['conn_frac'] = arbor_ais_df_pot['num_cells_chc'] / arbor_ais_df_pot['num_potential']
mask_frac_df = pd.read_hdf(data_dir + '/mask_fraction_data_v{}.hdf'.format(data_version))
ais_oid_within_limits = mask_frac_df[mask_frac_df['d_{}'.format(d_max)] < fraction_cutoff]['root_id']
within_unmasked = np.isin(arbor_ais_df_pot['post_pt_root_id'], ais_oid_within_limits)
arbor_ais_df_pot = arbor_ais_df_pot[within_unmasked]

In [ ]:
use_df = arbor_ais_df_pot 

x = 'num_potential'
y = 'syn_net_chc'
fig, ax = make_scatterplot(x, y, arbor_ais_df_pot,
                           figsize, plot_label_lookup, tick_dict,
                           xtick_int=tickintdict.get(x, True),
                           ytick_int=tickintdict.get(y, True),
                           xprecision=xprecision.get(x, 1))
ax.set_title(f'R = {nan_pearsonr(use_df[x], use_df[y])[0]:.2f}')
fig.savefig(fname=f'{plot_dir}/scatterplots_{y}_vs_{x}.pdf', bbox_inches="tight")

x = 'conn_frac'
y = 'syn_net_chc'
fig, ax = make_scatterplot(x, y, arbor_ais_df_pot,
                           figsize, plot_label_lookup, tick_dict,
                           xtick_int=tickintdict.get(x, True),
                           ytick_int=tickintdict.get(y, True),
                           xprecision=xprecision.get(x, 1))
ax.set_title(f'R = {nan_pearsonr(use_df[x], use_df[y])[0]:.2f}')
fig.savefig(fname=f'{plot_dir}/scatterplots_{y}_vs_{x}.pdf', bbox_inches="tight")

x = 'conn_frac'
y = 'num_potential'
fig, ax = make_scatterplot(x, y, arbor_ais_df_pot,
                           figsize, plot_label_lookup, tick_dict,
                           xtick_int=tickintdict.get(x, True),
                           ytick_int=tickintdict.get(y, True),
                           xprecision=xprecision.get(x, 1))
ax.set_title(f'R = {nan_pearsonr(use_df[x], use_df[y])[0]:.2f}')
fig.savefig(fname=f'{plot_dir}/scatterplots_{y}_vs_{x}.pdf', bbox_inches="tight")

In [ ]:
fraction_cutoff = 0.1
figsize = (2, 2)

n_pot_dist = {}
base_variables = ['pca_0', 'pca_1', 'pca_2']

for d_max in d_vec:

    is_potential = is_potential_ds[d_max]

    num_pot_df = pd.DataFrame({'pyc_root_id': complete_ais_ids, 'num_potential':[sum(is_potential[ii].values()) for ii in range(len(complete_ais_ids))]})
    arbor_ais_df_pot = arbor_ais_df_use.merge(num_pot_df, left_on='post_pt_root_id', right_on='pyc_root_id', how='left').drop(columns=['pyc_root_id'])
    arbor_ais_df_pot['conn_frac'] = arbor_ais_df_pot['num_cells_chc'] / arbor_ais_df_pot['num_potential']

    mask_frac_df = pd.read_hdf(data_dir + '/mask_fraction_data_v{}.hdf'.format(data_version))
    ais_oid_within_limits = mask_frac_df[mask_frac_df['d_{}'.format(d_max)] < fraction_cutoff]['root_id']
    within_unmasked = np.isin(arbor_ais_df_pot['post_pt_root_id'], ais_oid_within_limits)

    fig, ax = plt.subplots(figsize=figsize)
    ax.hist(arbor_ais_df_pot[within_unmasked]['conn_frac'], bins=np.arange(0,1.01,0.1), edgecolor='w', color='k')
    ax.grid(True, axis='y')
    ax.set_axisbelow(True)
    sns.despine(offset=2, trim=False, ax=ax)
    ax.set_xlabel('Connectivity fraction')
    ax.set_yticks(np.arange(0,31,5))
    ax.set_ylabel('# AISes')

    fig.savefig(fname=plot_dir+'/connectivity_fraction_histogram_dmax_{}.pdf'.format(d_max), bbox_inches="tight")

    conn_fract_described_df = arbor_ais_df_pot[within_unmasked]['conn_frac'].describe(percentiles=[0.5])
    conn_fract_described_df.to_csv(plot_dir + '/connectivity_fraction_summary_v{}_dmax_{}.csv'.format(data_version, d_max))

    
    n_pot_dist[d_max] = arbor_ais_df_pot[['post_pt_root_id', 'num_potential', 'num_cells_chc', 'size_net_chc']].rename(columns={'num_potential':'num_potential_{}'.format(d_max),
                                                                                             'num_cells_chc': 'num_cells_chc_{}'.format(d_max),
                                                                                             'size_net_chc': 'size_net_chc_{}'.format(d_max),
                                                                                             'post_pt_root_id': 'post_pt_root_id_{}'.format(d_max)})
    n_pot_dist[d_max]['within_data_{}'.format(d_max)] = within_unmasked
    
    common_y = ['soma_y_adj', 'n_syn_soma', 'syn_net_non', 'soma_x_um']
    if 'label' in arbor_ais_df.columns:
        common_y += ['n_syn_soma', 'soma_syn_density']

    new_col_pairs = {}
    new_col_pairs['conn_frac'] = common_y
    new_col_pairs['num_potential'] = common_y

    tickintdict['conn_frac'] = False

#     for x in new_col_pairs:
#         for y in new_col_pairs.get(x):
#             fig, ax = make_scatterplot(y, x, arbor_ais_df_pot[within_unmasked],
#                                        figsize, plot_label_lookup, tick_dict,
#                                        xtick_int=tickintdict.get(y, True),
#                                        ytick_int=tickintdict.get(x, True),
#                                        xprecision=xprecision.get(y, 1))
#             fig.savefig(fname=plot_dir+'/scatterplots_{x}_vs_{y}_dmax_{d}.pdf'.format(x=x, y=y, d=d_max), bbox_inches="tight")
    
    #### OLS plots
    
    for use_robust in [True, False]:
        y_col = 'conn_frac'
        columns_frac = base_variables
        fig, ax, res_df = ols_analysis_single(arbor_ais_df_pot, within_unmasked, y_col, columns_frac[::-1], conn_frac_color,
                                              robust=use_robust, xticks=np.arange(-0.75,.76,0.25), plot_label_lookup=plot_label_lookup)
        res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_factor_analysis_{y_col}_d{d_max}.csv'.format(y_col), index=False)
        fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_factor_analysis_{y_col}_d{d_max}.pdf', bbox_inches="tight")

        y_col = 'num_potential'
        columns_pot = base_variables
        fig, ax, res_df = ols_analysis_single(arbor_ais_df_pot, within_unmasked, y_col, columns_pot[::-1], num_pot_color,
                                              robust=use_robust, xticks=np.arange(-0.75,.76,0.25), plot_label_lookup=plot_label_lookup)
        res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_factor_analysis_{y_col}_d{d_max}.csv'.format(y_col), index=False)
        fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_factor_analysis_{y_col}_d{d_max}.pdf', bbox_inches="tight")

        for ycol in ['conn_frac', 'num_potential']:
            for xcol in base_variables:
                for use_robust in [True, False]:
                    fig, ax = plt.subplots(figsize=(3,3))
                    sns.regplot(x=xcol, y=ycol, data=arbor_ais_df_pot, ax=ax, marker='o', color='k', scatter_kws={'s':8, 'color':(0.3, 0.3, 0.3)}, robust=use_robust, n_boot=300)
                    sns.despine(ax=ax, offset=5)
                    ax.set_xlabel(plot_label_lookup[xcol])
                    ax.set_ylabel(plot_label_lookup[ycol])
                    r=nan_pearsonr(arbor_ais_df_pot[xcol], arbor_ais_df_pot[ycol])[0]
                    ax.set_title(f'R$^2$={r*r:.2f}')
                    fig.savefig(f'{plot_dir}/linear_plot_robust_{use_robust}_{ycol}_vs_{xcol}.pdf')
                    residual_scatterplots(ycol, base_variables, arbor_ais_df_pot['post_pt_root_id']>0, arbor_ais_df_pot, 'exact', plot_dir, plot_label_lookup=plot_label_lookup, robust=use_robust)


In [ ]:
out_df = n_pot_dist[d_vec[0]]
for dval in d_vec[1:]:
    out_df = out_df.merge(n_pot_dist[dval],left_index=True, right_index=True)

out_df = out_df.rename(columns = {'num_cells_chc_10000':'num_conn_chc', 'size_net_chc_10000':'size_net_chc', 'post_pt_root_id_10000':'pt_root_id'})
drop_columns = ['num_cells_chc_7500', 'size_net_chc_7500', 'post_pt_root_id_7500',
                'num_cells_chc_5000', 'size_net_chc_5000', 'post_pt_root_id_5000',
                'num_cells_chc_15000', 'size_net_chc_15000', 'post_pt_root_id_15000',]
out_df = out_df.drop(columns=drop_columns)


In [ ]:
out_df.to_csv(data_dir + '/out/potential_synapse_counts_truncated_v{}.csv'.format(data_version))

---
#### Input density

In [ ]:
dend_df = pd.read_hdf(base_dir+'/data/in/spatial_arbor_synapses_v{}.h5'.format(data_version), 'arbor_synapse_df')

In [ ]:
sns.scatterplot(x='dend_syn', y='dend_area', data=dend_df)

In [ ]:
dend_df['dend_syn_density'] = dend_df['dend_syn'] / dend_df['dend_area']

In [ ]:
arbor_ais_density_df = arbor_ais_df_pot.merge(dend_df[['pt_root_id', 'dend_syn_density']], left_on='post_pt_root_id', right_on='pt_root_id', how='inner')

In [ ]:
plot_label_lookup['dend_syn_density'] = 'Dend Syn. Density ($1/\mu m^2$)'
tick_dict['dend_syn_density'] = np.arange(0.19, 0.35, 0.03)

fig, ax = make_scatterplot('dend_syn_density', 'syn_net_chc', arbor_ais_density_df,
                           (2,2), plot_label_lookup, tick_dict,
                           xtick_int=False,
                           ytick_int=tickintdict.get(y, True),
                           xprecision=2)
fig.savefig(plot_dir + '/scatterplots_{}_v_{}.pdf'.format('dend_syn_density', 'syn_net_chc'), bbox_inches='tight')

In [ ]:
stats.pearsonr(arbor_ais_density_df['dend_syn_density'], arbor_ais_density_df['syn_net_chc'])

In [ ]:
plot_label_lookup['dend_syn_density'] = 'Dend Syn. Density ($1/\mu m^2$)'
tick_dict['dend_syn_density'] = np.arange(0.22, 0.32, 0.03)

fig, ax = make_scatterplot('dend_syn_density', 'syn_net_non', arbor_ais_density_df,
                           (2,2), plot_label_lookup, tick_dict,
                           xtick_int=False,
                           ytick_int=tickintdict.get(y, True),
                           xprecision=2)
fig.savefig(plot_dir + '/scatterplots_{}_v_{}.pdf'.format('dend_syn_density', 'syn_net_non'), bbox_inches='tight')

In [ ]:
arbor_ais_density_df.columns

---

In [ ]:
arbor_ais_df.columns

In [ ]:
plot_label_lookup['ais_radius'] = 'AIS rad ($\mu m$)'

In [ ]:
fraction_cutoff = 0.1
figsize = (2, 2)
use_robust = False

n_pot_dist = {}

for d_max in d_vec:

    is_potential = is_potential_ds[d_max]

    num_pot_df = pd.DataFrame({'pyc_root_id': complete_ais_ids, 'num_potential':[sum(is_potential[ii].values()) for ii in range(len(complete_ais_ids))]})
    arbor_ais_df_pot = arbor_ais_df_r.merge(num_pot_df, left_on='post_pt_root_id', right_on='pyc_root_id', how='left').drop(columns=['pyc_root_id'])
    arbor_ais_df_pot['conn_frac'] = arbor_ais_df_pot['num_cells_chc'] / arbor_ais_df_pot['num_potential']

    mask_frac_df = pd.read_hdf(data_dir + '/mask_fraction_data_v{}.hdf'.format(data_version))
    ais_oid_within_limits = mask_frac_df[mask_frac_df['d_{}'.format(d_max)] < fraction_cutoff]['root_id']
    within_unmasked = np.isin(arbor_ais_df_pot['post_pt_root_id'], ais_oid_within_limits)

    fig, ax = plt.subplots(figsize=figsize)
    ax.hist(arbor_ais_df_pot[within_unmasked]['conn_frac'], bins=np.arange(0,1.01,0.1), edgecolor='w', color='k')
    ax.grid(True, axis='y')
    ax.set_axisbelow(True)
    sns.despine(offset=2, trim=False, ax=ax)
    ax.set_xlabel('Connectivity fraction')
    ax.set_yticks(np.arange(0,31,5))
    ax.set_ylabel('# AISes')

    arbor_ais_df_pot['conn_frac'].describe()
    fig.savefig(fname=plot_dir+'/connectivity_fraction_histogram_dmax_{}.pdf'.format(d_max), bbox_inches="tight")

    conn_fract_described_df = arbor_ais_df_pot[within_unmasked]['conn_frac'].describe(percentiles=[0.5])
    conn_fract_described_df.to_csv(plot_dir + '/connectivity_fraction_summary_v{}_dmax_{}.csv'.format(data_version, d_max))

    
    n_pot_dist[d_max] = arbor_ais_df_pot[['post_pt_root_id', 'num_potential', 'num_cells_chc', 'size_net_chc']].rename(columns={'num_potential':'num_potential_{}'.format(d_max),
                                                                                             'num_cells_chc': 'num_cells_chc_{}'.format(d_max),
                                                                                             'size_net_chc': 'size_net_chc_{}'.format(d_max),
                                                                                             'post_pt_root_id': 'post_pt_root_id_{}'.format(d_max)})
    n_pot_dist[d_max]['within_data_{}'.format(d_max)] = within_unmasked
    
    common_y = ['soma_y_adj', 'n_syn_soma', 'syn_net_non', 'soma_x_um']
    if 'label' in arbor_ais_df.columns:
        common_y += ['soma_synapses', 'soma_syn_density']

    new_col_pairs = {}
    new_col_pairs['conn_frac'] = common_y
    new_col_pairs['num_potential'] = common_y

    tickintdict['conn_frac'] = False

    for x in new_col_pairs:
        for y in new_col_pairs.get(x):
            fig, ax = make_scatterplot(y, x, arbor_ais_df_pot[within_unmasked],
                                       figsize, plot_label_lookup, tick_dict,
                                       xtick_int=tickintdict.get(y, True),
                                       ytick_int=tickintdict.get(x, True),
                                       xprecision=xprecision.get(y, 1))
            fig.savefig(fname=plot_dir+'/scatterplots_{x}_vs_{y}_dmax_{d}.pdf'.format(x=x, y=y, d=d_max), bbox_inches="tight")
    
    #### OLS plots
    
    y_col = 'conn_frac'
    columns_frac =  ['n_syn_soma', 'soma_y_um', 'soma_x_um', 'syn_net_non', 'ais_radius', 'size_mean_chc']
    fig, ax, res_df = ols_analysis_single(arbor_ais_df_pot, row_filter, y_col, columns_frac[::-1], conn_frac_color,
                                          robust=use_robust, xticks=np.arange(-0.75,.76,0.25))
    res_df.to_csv(plot_dir + '/ols_fit_{}_dmax_{}.csv'.format(y_col, d_max), index=False)
    fig.savefig(plot_dir + '/ols_fit_{}_dmax_{}.pdf'.format(y_col, d_max), bbox_inches="tight")

    y_col = 'num_potential'
    columns_pot = ['n_syn_soma', 'soma_y_um', 'soma_x_um', 'syn_net_non', 'ais_radius', 'size_mean_chc']
    fig, ax, res_df = ols_analysis_single(arbor_ais_df_pot, row_filter, y_col, columns_pot[::-1], num_pot_color,
                                          robust=use_robust, xticks=np.arange(-0.75,.76,0.25))
    res_df.to_csv(plot_dir + '/ols_fit_{}_dmax_{}.csv'.format(y_col, d_max), index=False)
    fig.savefig(plot_dir + '/ols_fit_{}_dmax_{}.pdf'.format(y_col, d_max), bbox_inches="tight")


    if 'label' in arbor_ais_df.columns:
        y_col = 'conn_frac'
        columns_frac =  ['soma_synapses', 'soma_y_um', 'soma_x_um', 'syn_net_non', 'ais_radius', 'size_mean_chc']
        fig, ax, res_df = ols_analysis_single(arbor_ais_df_pot, row_filter, y_col, columns_frac[::-1], conn_frac_color,
                                              robust=use_robust, xticks=np.arange(-0.75,.76,0.25))
        res_df.to_csv(plot_dir + '/ols_fit_soma_exact_{}_dmax_{}.csv'.format(y_col, d_max), index=False)
        fig.savefig(plot_dir + '/ols_fit_soma_exact_{}_dmax_{}.pdf'.format(y_col, d_max), bbox_inches="tight")

        y_col = 'num_potential'
        columns_pot = ['soma_synapses', 'soma_y_um', 'soma_x_um', 'syn_net_non', 'ais_radius', 'size_mean_chc']
        fig, ax, res_df = ols_analysis_single(arbor_ais_df_pot, row_filter, y_col, columns_pot[::-1], num_pot_color,
                                              robust=use_robust, xticks=np.arange(-0.75,.76,0.25))
        res_df.to_csv(plot_dir + '/ols_fit_soma_exact_{}_dmax_{}.csv'.format(y_col, d_max), index=False)
        fig.savefig(plot_dir + '/ols_fit_soma_exact_{}_dmax_{}.pdf'.format(y_col, d_max), bbox_inches="tight")

        #### Soma synapse density
        y_col = 'conn_frac'
        columns_frac =  ['soma_syn_density', 'soma_y_um', 'soma_x_um', 'syn_net_non', 'ais_radius', 'size_mean_chc']
        fig, ax, res_df = ols_analysis_single(arbor_ais_df_pot, row_filter, y_col, columns_frac[::-1], conn_frac_color,
                                              robust=use_robust, xticks=np.arange(-0.75,.76,0.25))
        res_df.to_csv(plot_dir + '/ols_fit_soma_density_{}_dmax_{}.csv'.format(y_col, d_max), index=False)
        fig.savefig(plot_dir + '/ols_fit_soma_density_{}_dmax_{}.pdf'.format(y_col, d_max), bbox_inches="tight")

        y_col = 'num_potential'
        columns_pot = ['soma_syn_density', 'soma_y_um', 'soma_x_um', 'syn_net_non', 'ais_radius', 'size_mean_chc']
        fig, ax, res_df = ols_analysis_single(arbor_ais_df_pot, row_filter, y_col, columns_pot[::-1], num_pot_color,
                                              robust=use_robust, xticks=np.arange(-0.75,.76,0.25))
        res_df.to_csv(plot_dir + '/ols_fit_soma_density_{}_dmax_{}.csv'.format(y_col, d_max), index=False)
        fig.savefig(plot_dir + '/ols_fit_soma_density_{}_dmax_{}.pdf'.format(y_col, d_max), bbox_inches="tight")

    else:
        print('No exact soma data...')

In [ ]:
'ais_radius' in arbor_ais_df_pot.columns

---
### Functional data

In [ ]:
act_pup_df = pd.read_hdf(f'{base_dir}/notebooks/activity_df.h5', 'activity')

In [ ]:
dl = AnalysisDataLink(dataset_name=dataset_name,
                      sqlalchemy_database_uri=sql_database_uri_base,
                      materialization_version=data_version,
                      verbose=False)

somav1_df = dl.query_cell_types('soma_valence')

In [ ]:
act_pup_df2 = act_pup_df.merge(somav1_df[['id', 'pt_root_id']], left_on='soma_id', right_on='id', how='left').drop(columns=['id_x', 'id_y'])

In [ ]:
full_dat_df = arbor_ais_df_use.merge(act_pup_df2.drop(columns=['syn_net_chc']), left_on='post_pt_root_id', right_on='pt_root_id', how='left')

In [ ]:
from copy import copy
base_variables = [f'pca_{ii}' for ii in range(pca.n_components)]
plot_label_lookup_ext = copy(plot_label_lookup)
plot_label_lookup_ext['activity_during'] = 'activity_during'
plot_label_lookup_ext['activity_diff'] = 'activity_diff'
plot_label_lookup_ext['activity_outside'] = 'activity_outside'

y_col = 'syn_net_chc'
rel_variables = base_variables + ['activity_outside']
fig, ax, res_df = ols_analysis_single(full_dat_df, full_dat_df['post_pt_root_id']>0,
                                      y_col, rel_variables[::-1], chc_color, robust=False,
                                      plot_label_lookup=plot_label_lookup_ext,
                                      xticks=np.arange(-0.75,.76,0.25))



In [ ]:
relinds = full_dat_df['activity_diff']<0.1
nan_pearsonr(full_dat_df['pca_2'][relinds], full_dat_df['activity_diff'][relinds])

In [ ]:
plot_label_lookup['pca_1']

In [ ]:
sns.scatterplot(y='pca_1', x='activity_diff', data=full_dat_df)

In [ ]:
data_dir = base_dir 
pupil_data = f'{data_dir}/cell_pupil_metrics.csv'
pupil_df = pd.read_csv(pupil_data, header=1)

In [ ]:
648518346349538387 in soma_func_df['pt_root_id']

In [ ]:
soma_lookup_df = dl.query_cell_ids('functional_coregistration_raw')

In [ ]:
soma_lookup_df.query('func_id == 10')

In [ ]:
soma_func_df.query('func_id == 10')

In [ ]:
pupil_df.query('soma_id == 10')

In [ ]:
soma_func_df['func_id']

In [ ]:
soma_func_df = dl.query_cell_ids('functional_coregistration_lookup')

In [ ]:
soma_func_df.query('func_id==10')

In [ ]:
soma_func_df.query('pt_root_id==648518346349538384')

In [ ]:
soma_df.query('pt_root_id==648518346349538384')

In [ ]:
np.all( np.vstack(soma_func_df['pt_position'].values) == np.array([101597, 55229, 1412]), axis=1)

In [ ]:
soma_func_df.query('pt_root_id == 648518346349538384')

In [ ]:
con_fname = f'{data_dir}/con_onset_mean_activity.npy'
dil_fname = f'{data_dir}/dil_onset_mean_activity.npy'

run_on_fname = f'{data_dir}/run_onset_mean_activity.npy'
run_off_fname = f'{data_dir}/run_offset_mean_activity.npy'

In [ ]:
pupil_df['con_activity'] = [x for x in np.load(con_fname)]
pupil_df['dil_activity'] = [x for x in np.load(dil_fname)]
pupil_df['run_on_activity'] = [x for x in np.load(run_on_fname)]
pupil_df['run_off_activity'] = [x for x in np.load(run_off_fname)]

In [ ]:
keep_columns = ['pt_root_id', 'soma_id',
       'dj_soma_x', 'dj_soma_y', 'dj_soma_z',
       'mean_firing_dil', 'mean_firing_con', 'dcp_index',
       'mean_firing_dil.1', 'mean_firing_con.1', 'dcp_index.1',
       'mean_firing_dil.2', 'mean_firing_con.2', 'dcp_index.2',
       'mean_firing_dil.3', 'mean_firing_con.3', 'dcp_index.3',
       'dil_activity', 'con_activity', 'run_on_activity', 'run_off_activity']
pupil_df.rename(columns={'soma_x': 'dj_soma_x', 
                        'soma_y': 'dj_soma_y',
                        'soma_z': 'dj_soma_z',},
                inplace=True)

In [ ]:
dl = AnalysisDataLink(dataset_name=dataset_name,
                      sqlalchemy_database_uri=sql_database_uri_base,
                      materialization_version=data_version,
                      verbose=False)


In [ ]:
pupil_df

In [ ]:
soma_func_df = dl.query_cell_ids('functional_coregistration_lookup')
# soma_func_df = dl.query_cell_ids(soma_table)
# soma_pupil_df = soma_func_df.merge(pupil_df, left_on='id', right_on='soma_id')

soma_pupil_df = soma_func_df.merge(pupil_df, left_on='func_id', right_on='soma_id')

soma_pupil_df[['dj_soma_x', 'dj_soma_y', 'dj_soma_z', 'pt_position']]

In [ ]:
soma_func_df = dl.query_cell_ids('functional_coregistration_lookup')
# soma_func_df = dl.query_cell_ids('soma_valence_v2')
# soma_pupil_df = soma_func_df.merge(pupil_df, left_on='id', right_on='soma_id')

soma_pupil_df = soma_func_df.merge(pupil_df, left_on='func_id', right_on='soma_id')
soma_pupil_df = soma_pupil_df[keep_columns]

pupil_plus_df = arbor_ais_df_use.merge(soma_pupil_df, how='left', left_on='post_pt_root_id', right_on='pt_root_id')

sns.lmplot(x='soma_y', y='dcp_index', data=pupil_plus_df, robust=True)

syn_pupil_dat = pupil_plus_df[['syn_net_chc', 'mean_firing_con.2']].values
rows_keep = ~ np.any(np.isnan(syn_pupil_dat), axis=1)
syn_pupil_dat=syn_pupil_dat[rows_keep,:]

stats.spearmanr(syn_pupil_dat[:,0], syn_pupil_dat[:,1])

In [ ]:
soma_func_df = dl.query_cell_ids('soma_valence_v2')
soma_pupil_df = soma_func_df.merge(pupil_df, left_on='id', right_on='soma_id')

soma_pupil_df = soma_pupil_df[keep_columns]

pupil_plus_df = arbor_ais_df_use.merge(soma_pupil_df, how='left', left_on='post_pt_root_id', right_on='pt_root_id')

sns.lmplot(x='soma_y', y='dcp_index', data=pupil_plus_df, robust=True)

syn_pupil_dat = pupil_plus_df[['syn_net_chc', 'mean_firing_con.1']].values
rows_keep = ~ np.any(np.isnan(syn_pupil_dat), axis=1)
syn_pupil_dat=syn_pupil_dat[rows_keep,:]

stats.spearmanr(syn_pupil_dat[:,0], syn_pupil_dat[:,1])

In [ ]:
sns.scatterplot( x='mean_firing_dil.2', y='mean_firing_con.2', size='syn_net_chc', hue='syn_net_chc', data=pupil_plus_df)

In [ ]:
sns.scatterplot(x='soma_y', y='soma_id', data=pupil_plus_df)

In [ ]:
base_variables = [f'pca_{ii}' for ii in range(pca.n_components)]
plot_label_lookup_ext = copy(plot_label_lookup)
plot_label_lookup_ext['dcp_index'] = 'dcp_index'
plot_label_lookup_ext['corr'] = 'pupil_corr'

y_col = 'dcp_index'
rel_variables = ['syn_net_chc',  'soma_y_adj']
fig, ax, res_df = ols_analysis_single(pupil_plus_df, pupil_plus_df['post_pt_root_id']>0, y_col, rel_variables[::-1], chc_color, robust=False, plot_label_lookup=plot_label_lookup_ext, xticks=np.arange(-0.75,.76,0.25))

# if do_save:
#     res_df.to_csv(f'{plot_dir}/{ls_prefix[use_robust]}_fit_factor_analysis_{y_col}.csv'.format(y_col), index=False)
#     fig.savefig(f'{plot_dir}/{ls_prefix[use_robust]}_fit_factor_analysis_{y_col}.pdf'.format(y_col), bbox_inches="tight")
#     residual_scatterplots(y_col, columns_chc, row_filter_true, arbor_ais_df_use, 'exact', plot_dir, plot_label_lookup=plot_label_lookup, robust=use_robust)


In [ ]:
stats.spearmanr(pupil_plus_df[inds_rel]['dcp_index'], pupil_plus_df[inds_rel]['pca_2'])

In [ ]:
fig, ax = plt.subplots(figsize=(4,4) )
fig.set_facecolor('w')
sns.scatterplot(y='corr', x='syn_net_chc', data=pupil_plus_df, color='k', ax=ax)

In [ ]:
sns.scatterplot(x='dcp_index', y='syn_net_chc', data=pupil_plus_df)

In [ ]:
sns.lmplot(y='dcp_index', x='syn_net_chc', data=pupil_plus_df.query('soma_y>40000'), scatter_kws={'color':'k'} )

In [ ]:
pupil_plus_df.soma_y.describe()

In [ ]:
import jitterbar

In [ ]:
jitterbar.jitterbar?

In [ ]:
low_inds = pupil_plus_df['dcp_index']<np.nanmedian(pupil_plus_df['dcp_index'])
high_inds = pupil_plus_df['dcp_index']>=np.nanmedian(pupil_plus_df['dcp_index'])

In [ ]:
y_col = 'syn_net_chc'

cutoff = np.nanmean(pupil_plus_df['dcp_index'])
low_inds = pupil_plus_df['dcp_index']<=cutoff
high_inds = pupil_plus_df['dcp_index']>cutoff

fig, ax = plt.subplots(figsize=(2,3))
jitterbar.jitterbar([pupil_plus_df[low_inds][y_col], pupil_plus_df[high_inds][y_col]],
                    color=[[0,0,0], [0,0,0]], mode='mean', ax=ax, labels=['low dcp', 'high dcp'])
ax.set_ylabel(y_col)

In [ ]:
stats.ttest_ind(pupil_plus_df[high_inds][y_col], arbor_ais_df_use[y_col])

In [ ]:
stats.spearmanr(pupil_plus_df['syn_net_chc'], pupil_plus_df['dcp_index'], nan_policy='omit')

In [ ]:
ax.hlines?

In [ ]:
stats.fisher_exact?

In [ ]:
xcutoffs = np.arange(2,20)
ycutoffs = np.arange(-0.2, 0.6, 0.05)
table_pvalues = np.zeros((len(xcutoffs), len(ycutoffs)))
for ii, xcutoff in enumerate(xcutoffs):
    for jj, ycutoff in enumerate(ycutoffs):
        inds_low = pupil_plus_df[y_col] < xcutoff
        
        low_data = pupil_plus_df[inds_low]['dcp_index']
        low_data = low_data[~np.isnan(low_data)]

        high_data = pupil_plus_df[~inds_low]['dcp_index']
        high_data = high_data[~np.isnan(high_data)]
        
        count_low_low = np.sum(low_data < ycutoff)
        count_low_high = np.sum(low_data >= ycutoff)
        table_low = [count_low_low, count_low_high]
        
        count_high_low = np.sum(high_data < ycutoff)
        count_high_high = np.sum(high_data >= ycutoff)
        table_high = [count_high_low, count_high_high]

        oddsratio, pv = stats.fisher_exact([table_low, table_high])
        table_pvalues[ii, jj] = pv

In [ ]:
plt.plot()

In [ ]:
inds_rel = ~np.isnan(pupil_plus_df['corr'])
xcutoff = np.nanmedian(pupil_plus_df[inds_rel]['corr'])
ycutoff = np.nanmedian(pupil_plus_df[inds_rel]['syn_net_chc'])

count_low_low = np.sum(pupil_plus_df[inds_rel].query(f'corr <= {xcutoff}')['syn_net_chc'] <= ycutoff)
count_low_high = np.sum(pupil_plus_df[inds_rel].query(f'corr <= {xcutoff}')['syn_net_chc'] > ycutoff)
count_high_low = np.sum(pupil_plus_df[inds_rel].query(f'corr > {xcutoff}')['syn_net_chc'] <= ycutoff)
count_high_high = np.sum(pupil_plus_df[inds_rel].query(f'corr > {xcutoff}')['syn_net_chc'] > ycutoff)

table_low = [count_low_low, count_low_high]
table_high = [count_high_low, count_high_high]

stats.fisher_exact([table_low, table_high])

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
sns.scatterplot(x='corr', y='syn_net_chc', data=pupil_plus_df[inds_rel], ax=ax)
ax.hlines(ycutoff, -0.2, 0.25)
ax.vlines(xcutoff, 0, 25 )

In [ ]:
stats.pearsonr(pupil_plus_df[inds_rel]['dcp_index'], pupil_plus_df[inds_rel][''])

In [ ]:
sns.scatterplot(x='dcp_index', y='pca_2', data=pupil_plus_df[inds_rel])
sns.scatterplot(np.zeros(len(arbor_ais_df_use)), arbor_ais_df_use['pca_2'])

In [ ]:
table_high

In [ ]:
fig, ax = plt.subplots(figsize=(3,3))
ax.matshow(table_pvalues < 0.03)

In [ ]:
np.nanmedian(pupil_plus_df['dcp_index'])

In [ ]:
xcutoffs = np.arange(-0.1, 0.333, 0.05)
y_col = 'dcp_index'

pvalues = []
for xcutoff in xcutoffs:
    inds_low = pupil_plus_df[y_col] < xcutoff
    low_data = pupil_plus_df[inds_low]['syn_net_chc']
    low_data = low_data[~np.isnan(low_data)]

    inds_high = pupil_plus_df[y_col] > xcutoff
    high_data = pupil_plus_df[inds_high]['syn_net_chc']
    high_data = high_data[~np.isnan(high_data)]

    out = stats.ttest_ind(low_data, high_data)
    pvalues.append(out.pvalue)

fig, ax = plt.subplots(figsize=(4,4))    
ax.plot(xcutoffs, pvalues, 's-')
ax.hlines(0.05, np.min(xcutoffs), np.max(xcutoffs),  linestyle='--')
ax.set_yscale('log')
ax.set_xlabel('DCP Index Cutoff')
ax.set_ylabel('p-value for mean chc syn')

In [ ]:
xcutoffs = np.arange(2,20)
y_col = 'syn_net_chc'

pvalues = []
for xcutoff in xcutoffs:
    inds_low = pupil_plus_df[y_col] < xcutoff
    low_data = pupil_plus_df[inds_low]['dcp_index']
    low_data = low_data[~np.isnan(low_data)]

    high_data = pupil_plus_df[~inds_low]['dcp_index']
    high_data = high_data[~np.isnan(high_data)]

    out = stats.ttest_ind(low_data, high_data)
    pvalues.append(out.pvalue)


fig, ax = plt.subplots(figsize=(4,4))    
ax.plot(xcutoffs, pvalues, 's-')
ax.hlines(0.05, np.min(xcutoffs), np.max(xcutoffs),  linestyle='--')
ax.set_yscale('log')

In [ ]:
pupil_plus_df[inds_low]['dcp_index']

In [ ]:
sns.scatterplot(y='dcp_index', x='soma_y', data=pupil_plus_df)

In [ ]:
sns.scatterplot(y='corr', x='syn_net_chc', data=pupil_plus_df)

In [ ]:
corr_matrix_columns = [
 'soma_synapses',
 'soma_area',
 'soma_syn_density',
 'syn_net_non',
 'ais_radius',
 'soma_y_um',
 'dcp_index',
 'corr']

extra_corr_matrix_columns = ['syn_net_chc']+corr_matrix_columns
ais_item_data = pupil_plus_df[inds_rel][extra_corr_matrix_columns].fillna(0)

In [ ]:
corr_mat = np.zeros((len(extra_corr_matrix_columns), len(extra_corr_matrix_columns)))
corr_mat_p = np.zeros((len(extra_corr_matrix_columns), len(extra_corr_matrix_columns)))
for ii in range(len(extra_corr_matrix_columns)):
    for jj in range(len(extra_corr_matrix_columns)):
        r, p = nan_pearsonr(pupil_plus_df[extra_corr_matrix_columns[ii]], pupil_plus_df[extra_corr_matrix_columns[jj]])
        corr_mat[ii,jj] = r
        corr_mat_p[ii,jj] = p

In [ ]:
from statsmodels.stats import multitest
is_sig, corr_p, _, _ = multitest.multipletests(corr_mat_p[np.tril_indices_from(corr_mat, k=-1)])
tri_inds = np.tril_indices_from(corr_mat, k=-1)
put_star = []
for ii, jj, sig in zip(*tri_inds, is_sig):
    if sig:
        put_star.append([ii, jj])
put_star = np.array(put_star)

In [ ]:
do_plot = True 

mask = np.zeros_like(corr_mat, dtype=np.bool)
mask[np.tril_indices_from(mask)] = True

cmap = sns.color_palette('RdBu', n_colors=31)
fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(corr_mat, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)
ax.plot(put_star[:,0]+0.5, put_star[:,1]+0.5, 'k*')
ax.set_yticks(np.arange(0.5, len(extra_corr_matrix_columns)-1+0.5))
_=ax.set_yticklabels([plot_label_lookup_ext[x] for x in extra_corr_matrix_columns[:-1]], rotation=0)

ax.set_xticks(np.arange(1.5, len(extra_corr_matrix_columns)+0.5))
_=ax.set_xticklabels([plot_label_lookup_ext[x] for x in extra_corr_matrix_columns[1:]], rotation=45)
ax.xaxis.tick_top()


In [ ]:
ais_item_data = pupil_plus_df[inds_rel][corr_matrix_columns].fillna(0)
Xdat = ais_item_data.values
Xz = stats.zscore(Xdat, axis=0)

# pca = FastICA(n_components=3, random_state=5010 )
pca = FactorAnalysis(n_components=4)
Xz_pca = pca.fit_transform(Xz)

for ii in range(pca.n_components):
    if pca.components_[ii,np.argmax(np.abs(pca.components_[ii,:]))] < 0:   # If the dominant component is negative
        pca.components_[ii, :] = -1 * pca.components_[ii, :]
        Xz_pca[:,ii] = -1 * Xz_pca[:,ii]

for ii in range(pca.n_components):
    fig, ax = plt.subplots(figsize=(1,2))
    ax.barh(np.arange(pca.components_.shape[1]), pca.components_[ii,:], height=0.5, color='k')
    ax.vlines(0, -1, 7, linewidth=1)
    ax.set_ylim((-0.5, 7.5))
    ax.set_yticks(np.arange(pca.components_.shape[1]))
    ax.set_yticklabels([plot_label_lookup_ext[x] for x in corr_matrix_columns])
    ax.invert_yaxis()
    maxval=np.max(np.abs(pca.components_[ii,:]))
    ax.set_xlim((-maxval, maxval))
    ax.set_xticks((-maxval, 0, maxval))
    ax.set_xticklabels((f'{-maxval:0.2f}', '0', f'{maxval:0.2f}'))
    sns.despine(ax=ax, offset=5, trim=True)
    ax.set_title(f'PC {ii}')


In [ ]:
fig, ax = plt.subplots(figsize=(3,3))
sns.scatterplot(x='corr', y='dcp_index', data=pupil_plus_df, ax=ax)